# Numpy LSTM

## Code from Max

In [ ]:
import numpy as np

In [ ]:
class Module(object):
    def __call__(self, *args):
        return self.forward(*args)

    def forward(self, *args):
        raise NotImplementedError("Please overwrite.")

class Parameter(object):
    def __init__(self, array):
        self.array = array
        self.grad = np.zeros_like(array)
    
    def reset_gradients(self):
        self.grad = np.zeros_like(self.array)

class Tanh(Module):
    def forward(self, x):
        return (np.exp(2 * x) - 1) / (np.exp(2 * x) + 1)

class Sigmoid(Module):
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

class LSTMCell(Module):
    def __init__(self, input_size=10, hidden_size=128):
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.W = Parameter(np.random.randn(input_size, 4 * hidden_size))
        self.U = Parameter(np.random.randn(hidden_size, 4 * hidden_size))
        self.bias = Parameter(np.random.randn(4 * hidden_size))

        self.tanh = Tanh()
        self.sigmoid = Sigmoid()
        
        self.backprop_cache = {}

    def forward(self, x, h=None, c=None):
        if h is None:
            h = np.zeros((x.shape[0], self.hidden_size))
        if c is None:
            c = np.zeros((x.shape[0], self.hidden_size))
            
        at = np.dot(x, self.W.array) + np.dot(h, self.U.array) + self.bias.array

        ft = self.sigmoid(at[:, :self.hidden_size])
        it = self.sigmoid(at[:, self.hidden_size:(self.hidden_size*2)])
        ot = self.sigmoid(at[:, (self.hidden_size*2):(self.hidden_size*3)])
        zt = self.tanh(at[:, (self.hidden_size*3):])
        ct = ft * c + it * zt
        ht = ot * self.tanh(ct)
        
        self.backprop_cache = {
            'x': x,
            'W': self.W,
            'U': self.U,
            'bias': self.bias,
            'h': h,
            'c': c,
            'ft': ft,
            'it': it,
            'ot': ot,
            'ct': ct,
            'zt': zt,
            'ht': ht,
            'at': at
        }
        return ht

    def backward(self, above_y=None, above_dh=None, above_dc=None):
        # TODO : make loss not just linear
        d_err_net = -1
        d_net_ct = tanh_backward(self.backprop_cache['ct']) * self.backprop_cache['ot'] * d_err_net
        d_net_ot = self.backprop_cache['ct'] * d_err_net
        d_ct_ft = self.backprop_cache['c'] * d_net_ct
        d_ct_it = self.backprop_cache['zt'] * d_net_ct
        d_ct_zt = self.backprop_cache['it'] * d_net_ct
        d_ft_at = sig_backward(self.backprop_cache['at'][:, :self.hidden_size]) * d_ct_ft
        d_it_at = sig_backward(self.backprop_cache['at'][:, self.hidden_size:(self.hidden_size*2)]) * d_ct_it
        d_ot_at = sig_backward(self.backprop_cache['at'][:, (self.hidden_size*2):(self.hidden_size*3)]) * d_net_ot
        d_zt_at = tanh_backward(self.backprop_cache['at'][:, (self.hidden_size*3):]) * d_ct_zt
        d_at = np.concatenate([d_ft_at, d_it_at, d_ot_at, d_zt_at], axis=-1)
        d_at_W = np.repeat(np.expand_dims(self.backprop_cache['x'], axis=-1), self.hidden_size * 4, axis=-1) * \
            np.expand_dims(d_at, axis=1)
        d_at_U = np.repeat(np.expand_dims(self.backprop_cache['h'], axis=-1), self.hidden_size * 4, axis=-1) * \
            np.expand_dims(d_at, axis=1)
        d_at_bias = self.backprop_cache['at'] * d_at
        
        # Update gradients
        self.W.grad += d_at_W.mean(axis=0)
        self.U.grad += d_at_U.mean(axis=0) 
        self.bias.grad += d_at_bias.mean(axis=0)

        return d_at_W, d_at_U, d_at_bias

    def numerical_grad(self, x, y, h=None, c=None, eps=1e-4, ix1=4, ix2=10):
        # run forward
        if h is None:
            h = np.zeros((x.shape[0], self.hidden_size))
        if c is None:
            c = np.zeros((x.shape[0], self.hidden_size))

        # test only one W
        Wplus = self.W.array.copy()
        Wplus[ix1, ix2] += eps
        at = np.dot(x, Wplus) + np.dot(h, self.U.array) + (self.bias.array)
        ft = self.sigmoid(at[:, :self.hidden_size])
        it = self.sigmoid(at[:, self.hidden_size:(self.hidden_size*2)])
        ot = self.sigmoid(at[:, (self.hidden_size*2):(self.hidden_size*3)])
        zt = self.tanh(at[:, (self.hidden_size*3):])
        ct = ft * c + it * zt
        ht = ot * self.tanh(ct)
        # compute loss
        loss_1 = np.sum(y - ht, axis=1)
        
        Wminus = self.W.array.copy()
        Wminus[ix1, ix2] -= eps
        at = np.dot(x, Wminus) + np.dot(h, self.U.array) + (self.bias.array)
        ft = self.sigmoid(at[:, :self.hidden_size])
        it = self.sigmoid(at[:, self.hidden_size:(self.hidden_size*2)])
        ot = self.sigmoid(at[:, (self.hidden_size*2):(self.hidden_size*3)])
        zt = self.tanh(at[:, (self.hidden_size*3):])
        ct = ft * c + it * zt
        ht = ot * self.tanh(ct)
        # compute loss
        loss_2 = np.sum(y - ht, axis=1)
        
        # compute gradient approx
        return (loss_1 - loss_2) / (2 * eps)
    
    def reset_gradients(self):
        self.W.reset_gradients()
        self.U.reset_gradients()
        self.bias.reset_gradients()


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sig_backward(x):
    return sigmoid(x) * (1 - sigmoid(x))


def tanh_backward(x):
    return 1 - (np.exp(x) - np.exp(-x)) ** 2 / (np.exp(x) + np.exp(-x)) ** 2


In [ ]:
lstm = LSTMCell()

### Gradient Check on LSTM Cell

In [ ]:
x = np.ones((32, 10))
yhat = lstm(x)
lstm.reset_gradients()
dw, du, db = lstm.backward(np.ones_like(yhat))

In [ ]:
print(lstm.W.grad)

In [ ]:
ix1 = 9
ix2 = 405
print(dw[:, ix1, ix2])
print(lstm.numerical_grad(x, np.ones_like(yhat), ix1=ix1, ix2=ix2, eps=1e-6))